## Part 1

In [283]:
%reset -f

This part of the notebook is structured in the following way :<br>
question<br>
cell with function<br>
cell with exemple use of the function

In [284]:
import random
random.seed=0

### 1)
To represent the player we use the class Player

In [285]:
class Player:
    def __init__(self,playerId,score):
        self.id=playerId
        self.score=score #average score
        self.nb_games=0  #to calculate average
        self.left=None
        self.right=None
        self.height=1
        
    def __str__(self):
        return f"{self.id} : {round(self.score,3)} x {self.nb_games}"
    
    #ajouter un score
    def add_score(self,game_score):
        self.score=(self.score*self.nb_games+game_score)/(self.nb_games+1) #update average
        self.nb_games+=1 
        
    def reset_score(self):
        self.score=0
        self.nb_games=0
        
    def reset_childrens(self): #for re inserting in new tree
        self.right=None
        self.left=None
        self.heigth=1
    
    # display from https://stackoverflow.com/questions/34012886/print-binary-tree-level-by-level-in-python
    def display(self):
        lines, *_ = self._display_aux()
        for line in lines:
            print(line)

    def _display_aux(self):
        """Returns list of strings, width, height, and horizontal coordinate of the root."""
        # No child.
        if self.right is None and self.left is None:
            line = '%s' % round(self.score,2)
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle

        # Only left child.
        if self.right is None:
            lines, n, p, x = self.left._display_aux()
            s = '%s' % round(self.score,2)
            u = len(s)
            first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s
            second_line = x * ' ' + '/' + (n - x - 1 + u) * ' '
            shifted_lines = [line + u * ' ' for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2

        # Only right child.
        if self.left is None:
            lines, n, p, x = self.right._display_aux()
            s = '%s' % round(self.score,2)
            u = len(s)
            first_line = s + x * '_' + (n - x) * ' '
            second_line = (u + x) * ' ' + '\\' + (n - x - 1) * ' '
            shifted_lines = [u * ' ' + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

        # Two children.
        left, n, p, x = self.left._display_aux()
        right, m, q, y = self.right._display_aux()
        s = '%s' % round(self.score,2)
        u = len(s)
        first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y) * ' '
        second_line = x * ' ' + '/' + (n - x - 1 + u + y) * ' ' + '\\' + (m - y - 1) * ' '
        if p < q:
            left += [n * ' '] * (q - p)
        elif q < p:
            right += [m * ' '] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * ' ' + b for a, b in zipped_lines]
        return lines, n + m + u, max(p, q) + 2, n + u // 2




In [286]:
#exemple use :
print("single player :")
player=Player(1,0)
print(player)
print("same player after one game : ")
player.add_score(3)
print(player)

print("\ndisplay :")
p1=Player(1,3)
p2=Player(2,1)
p3=Player(3,10)
p1.left=p2
p1.right=p3
p1.display()


single player :
1 : 0 x 0
same player after one game : 
1 : 3.0 x 1

display :
 3_ 
/  \
1 10


### 2)
we use an AVL tree ranked by score
At each step of the game the player are mooved to a new tree with a new ranking

In [287]:
class Classement:
    
    #recursive insertion
    def insert(self,root,player):
        #insertion BST
        if not root:
            return(player)
        elif player.score<root.score:
            root.left=self.insert(root.left,player)
        elif player.score>root.score:
            root.right=self.insert(root.right,player)
        elif player.score==root.score: #if equality insert on lowest height side
            if self.getHeight(root.left)>self.getHeight(root.right):
                root.right=self.insert(root.right,player)
            elif self.getHeight(root.left)<self.getHeight(root.right):
                root.left=self.insert(root.left,player)
            elif self.getHeight(root.left)==self.getHeight(root.right): #if equal height choose random side
                side=random.choice(["left","right"])
                if side =="right":
                    root.right=self.insert(root.right,player)
                else :
                    root.left=self.insert(root.left,player)
                
        
        #height update
        root.height=1+max(self.getHeight(root.left),self.getHeight(root.right))
        
        #balance
        balance = self.getBalance(root)
        
        #balancing if necesssary
        if balance > 1 and player.score < root.left.score: 
            return self.rightRotate(root)
        if balance < -1 and player.score > root.right.score: 
            return self.leftRotate(root) 
        if balance > 1 and player.score > root.left.score: 
            root.left = self.leftRotate(root.left) 
            return self.rightRotate(root)
        if balance < -1 and player.score < root.right.score: 
            root.right = self.rightRotate(root.right) 
            return self.leftRotate(root) 
        
        return root
    
    def delete(self, root, player): 
  
        # BST delete 
        if not root: 
            return root 
        elif player.score < root.score: 
            root.left = self.delete(root.left, player) 
        elif player.score > root.score: 
            root.right = self.delete(root.right, player) 
        elif player.score == root.score :
            if player.id != root.id: #si il y a égalité on recherche des 2 cotés 
                root.right = self.delete(root.right, player) 
                root.left = self.delete(root.left, player) 
            else : 
                if root.left is None: 
                    temp = root.right 
                    root = None
                    return temp 

                elif root.right is None: 
                    temp = root.left 
                    root = None
                    return temp 

                temp = self.getMinPlayer(root.right) 
                root.score = temp.score
                root.id=temp.id
                root.nb_games=temp.nb_games
                root.right = self.delete(root.right,temp) 
            
                
        if root is None: 
            return root 
  
        # height update
        root.height = 1 + max(self.getHeight(root.left),self.getHeight(root.right)) 
  
        # balance 
        balance = self.getBalance(root) 
  
        if balance > 1 and self.getBalance(root.left) >= 0: 
            return self.rightRotate(root) 
        if balance < -1 and self.getBalance(root.right) <= 0: 
            return self.leftRotate(root)  
        if balance > 1 and self.getBalance(root.left) < 0: 
            root.left = self.leftRotate(root.left) 
            return self.rightRotate(root) 
        if balance < -1 and self.getBalance(root.right) > 0: 
            root.right = self.rightRotate(root.right) 
            return self.leftRotate(root) 
        
        return root 
    
    def leftRotate(self, z): 
        y = z.right 
        T2 = y.left 
        y.left = z 
        z.right = T2 
        z.height = 1 + max(self.getHeight(z.left),self.getHeight(z.right)) 
        y.height = 1 + max(self.getHeight(y.left),self.getHeight(y.right)) 
        return y 
    
    def rightRotate(self, z): 
        y = z.left 
        T3 = y.right 
        y.right = z 
        z.left = T3  
        z.height = 1 + max(self.getHeight(z.left),self.getHeight(z.right)) 
        y.height = 1 + max(self.getHeight(y.left),self.getHeight(y.right)) 
        return y 
        
    def getHeight(self, root): 
        if not root: 
            return 0
        return root.height 
    
    def getBalance(self, root): 
        if not root: 
            return 0
        return self.getHeight(root.left) - self.getHeight(root.right) 
    
    def getMinPlayer(self, root): 
        if root is None or root.left is None: 
            return root 
        return self.getMinPlayer(root.left) 
    
    def preOrder(self, root): 
        if not root: 
            return
        print(root) 
        self.preOrder(root.left) 
        self.preOrder(root.right) 
        
    def inOrder(self,root): 
        if not root: 
            return
        self.inOrder(root.left) 
        print(root)
        self.inOrder(root.right) 
        
    def returnInOrder(self,root): 
        if not root: 
            return
        self.inOrder(root.left) 
        return(root)
        self.inOrder(root.right) 

In [288]:
#exemple use :
tree=Classement()
print("Insertion : ")
root=None
print("Insertion Order : ")
for i in range(1,11):
    p=Player(i,0)
    p.add_score(i)
    print(f"Player : {p}")
    root=tree.insert(root,p)
root.display()

print("\nDeletion :")
print(f"deleting root ({root})")
root=tree.delete(root,root)
root.display()

print("\nIn order display :")
tree.inOrder(root)

Insertion : 
Insertion Order : 
Player : 1 : 1.0 x 1
Player : 2 : 2.0 x 1
Player : 3 : 3.0 x 1
Player : 4 : 4.0 x 1
Player : 5 : 5.0 x 1
Player : 6 : 6.0 x 1
Player : 7 : 7.0 x 1
Player : 8 : 8.0 x 1
Player : 9 : 9.0 x 1
Player : 10 : 10.0 x 1
     ____4.0__________         
    /                 \        
  _2.0_          ____8.0_      
 /     \        /        \     
1.0   3.0     _6.0_     9.0__  
             /     \         \ 
            5.0   7.0      10.0

Deletion :
deleting root (4 : 4.0 x 1)
     ____5.0_______         
    /              \        
  _2.0_       ____8.0_      
 /     \     /        \     
1.0   3.0   6.0_     9.0__  
                \         \ 
               7.0      10.0

In order display :
1 : 1.0 x 1
2 : 2.0 x 1
3 : 3.0 x 1
5 : 5.0 x 1
6 : 6.0 x 1
7 : 7.0 x 1
8 : 8.0 x 1
9 : 9.0 x 1
10 : 10.0 x 1


### 3)

In [289]:
#simulate score for a game of 10 players
#first generate a game with 10 players then assign the score to each players
def randGame(players_list):   
    # creating the lists:
    #2 impostors
    impostors=[0]*2  
    #the rest are crewmates
    crewmates=[0]*8
    
    #outcome of the game with a 50/50% chance for each teams
    impostorsWin=random.choice([True,False])   
    if impostorsWin:
        # impostors points :
        # +10 points for wining the game
        impostors=[player+10 for player in impostors]
        # +1 point per kill
        nbKillsImpostor1=random.randint(0,8) #game can be won without killing everybody
        nbKillsImpostor2=random.randint(0,8-nbKillsImpostor1)
        impostors[0]+=nbKillsImpostor1
        impostors[1]+=nbKillsImpostor2
        # +3 points per undiscovered murder 
        nbUndicoImpostor1=random.randint(0,nbKillsImpostor1)
        nbUndicoImpostor2=random.randint(0,nbKillsImpostor2)
        impostors[0]+=nbUndicoImpostor1*3
        impostors[1]+=nbUndicoImpostor2*3
        
        # crewmate points :
        # +3 points per unmasked imposter for 1, only 1 imposter can be unmasked because the impostors won
        unmaskedImpostor=random.choice([True,False])
        if unmaskedImpostor:
            crewmates[random.randrange(8)]+=3 
        # +1 point if all solo task made for all crew mates except at least one
        noPoints=random.randrange(8)
        crewmates=[player+(random.choice([0,1])) for player in (crewmates[:noPoints]+crewmates[noPoints+1:])]+[crewmates[noPoints]]
        
    else:
        #impostors points :
        # +1 point per kill
        nbKillsImpostor1=random.randint(0,7)
        nbKillsImpostor2=random.randint(0,7-nbKillsImpostor1)
        impostors[0]+=nbKillsImpostor1
        impostors[1]+=nbKillsImpostor2
        # +3 points per undiscovered murder 
        nbUndicoImpostor1=random.randint(0,nbKillsImpostor1)
        nbUndicoImpostor2=random.randint(0,nbKillsImpostor2)
        impostors[0]+=nbUndicoImpostor1*3
        impostors[1]+=nbUndicoImpostor2*3
    
        # Crewmates points :
        # +5 points for wining the game
        crewmates=[player+5 for player in crewmates]
        
        # 2 ways to win for crewmates : 
        # game can be won by completing all the tasks
        # or by discovering impostors
        win_task=random.choice([True,False]) 
        if win_task : #if the game was won by completing all tasks
            # +3 points per unmasked imposter for up to 1 player
            crewmates[random.randrange(8)]+=random.choice([0,3])      
            # +1 point if all solo task made for every players
            crewmates=[player+1 for player in crewmates]
        else : # if won by eliminating impostors
            # +3 points per unmasked imposter for up to 2 player
            crewmates[random.randrange(8)]+=3
            crewmates[random.randrange(8)]+=3
            # +1 point if all solo task made for all crew mates except at least one
            noPoints=random.randrange(8)
            crewmates=[player+(random.choice([0,1])) for player in (crewmates[:noPoints]+crewmates[noPoints+1:])]+[crewmates[noPoints]]
            
    #assigning new scores :
    scores_list=impostors+crewmates
    scores_list=[12 if score>12 else score for score in scores_list]
    [player.add_score(scores_list.pop(random.randrange(len(scores_list)))) for player in players_list]
    
    return  players_list

In [290]:
#used to generate n player to test the randGame function
def playersGen(n,scoreMax=0): #generate n players
    players =[]
    for i in range(n):
        if scoreMax:
            score=random.randint(0,scoreMax)
        else :
            score = 0
        players.append(Player(i,score))
    return players

In [291]:
#exemple use, playing one game with 10 players
players=playersGen(10)
players=randGame(players)
tree=Classement()
root=None
print("Players list after game : ")
[print(player) for player in players ];

Players list after game : 
0 : 0.0 x 1
1 : 1.0 x 1
2 : 0.0 x 1
3 : 1.0 x 1
4 : 10.0 x 1
5 : 4.0 x 1
6 : 0.0 x 1
7 : 0.0 x 1
8 : 12.0 x 1
9 : 0.0 x 1


### 5)
plays n games :

In [292]:
def nGames(players,n):
    for i in range(n):
        players=randGame(players)
        #print("game : " ,i)
        #[print(player) for player in players ]
    return players

In [293]:
#exemple use playing 100 games for a batch of 10 players and inserting them in tree
players=nGames(playersGen(10),100)
print("Players list after game")
[print(p) for p in players ]
root=None
for p in players:
    root=tree.insert(root,p)
print("\nPlayers in order in tree : ")
tree.inOrder(root)
print("\nTree:")
root.display()

Players list after game
0 : 4.24 x 100
1 : 4.65 x 100
2 : 4.51 x 100
3 : 5.44 x 100
4 : 4.28 x 100
5 : 4.76 x 100
6 : 4.24 x 100
7 : 4.86 x 100
8 : 4.69 x 100
9 : 4.56 x 100

Players in order in tree : 
6 : 4.24 x 100
0 : 4.24 x 100
4 : 4.28 x 100
2 : 4.51 x 100
9 : 4.56 x 100
1 : 4.65 x 100
8 : 4.69 x 100
5 : 4.76 x 100
7 : 4.86 x 100
3 : 5.44 x 100

Tree:
       _____4.51______________          
      /                       \         
   _4.24__             _____4.76______  
  /       \           /               \ 
4.24    4.28       _4.65__         _5.44
                  /       \       /     
                4.56    4.69    4.86    


### 4) 6) 7) <br>
manche1(n) generates the players, plays the first round and insert them in first tree <br>
manche(root_preced,nb_games=3,final=False)delete last 10 players, extract player from database, play random games based on ranking, re insert them in new updated database

In [294]:
# at each step of the game the player are removed by batch of 10
# Play their game and are then re inserted in a new tree for the next step
def manche1(n):
    tree = Classement()
    root=None
    for i in range(n): # 10 batchs 
        # création des joueurs
        players=[Player(i*10+j+1,0) for j in range (10)]
        # 3 premières parties 
        players=nGames(players,3)
        # insertion dans l'arbre 
        for player in players:
            root=tree.insert(root,player)
    #tree.inOrder(root)
    #root.display()
    return root


In [295]:
#exemple use, creating 3x10 players:
tree=Classement()
tree.inOrder(manche1(3))

12 : 2.667 x 3
1 : 3.333 x 3
22 : 3.667 x 3
13 : 3.667 x 3
25 : 3.667 x 3
29 : 4.0 x 3
21 : 4.0 x 3
27 : 4.0 x 3
18 : 4.0 x 3
5 : 4.0 x 3
24 : 4.0 x 3
10 : 4.0 x 3
28 : 4.0 x 3
19 : 4.333 x 3
14 : 4.333 x 3
16 : 4.333 x 3
7 : 4.667 x 3
2 : 4.667 x 3
20 : 4.667 x 3
9 : 5.333 x 3
23 : 5.333 x 3
4 : 5.333 x 3
15 : 6.0 x 3
8 : 6.0 x 3
30 : 7.333 x 3
3 : 7.333 x 3
17 : 7.667 x 3
6 : 7.667 x 3
11 : 8.333 x 3
26 : 9.0 x 3


In [296]:
def manche(root_preced,nb_games=3,final=False):
    tree=Classement()
    
    if not final:     #eliminate 10 last players 
        for i in range(10):
            minPlayer=tree.getMinPlayer(root_preced)
            root_preced=tree.delete(root_preced,minPlayer)

    root_new=None
    tree2=Classement()
    while(root_preced is not None):
        batch=[]
        #creating new games by batch of 10 from ranking
        for i in range(10):
            #print("root",root_preced.right,root_preced.left)
            minPlayer=tree.getMinPlayer(root_preced)
            root_preced=tree.delete(root_preced,minPlayer)
            if final:
                minPlayer.reset_score()
            batch.append(minPlayer)
        batch=nGames(batch,nb_games)
        
        #inserting player in new tree
        for player in batch:
            player.reset_childrens()
            try: 
                root_new=tree.insert(root_new,player)
            except :
                pass
    return root_new

In [300]:
#exemple use with 30 players :
tree=Classement()
root=manche1(3)
for _ in range(2):
    tree.inOrder(root)
    print("\n")
    root=manche(root)
tree.inOrder(root)

26 : 0.333 x 3
22 : 2.0 x 3
11 : 2.0 x 3
27 : 2.0 x 3
13 : 2.0 x 3
24 : 2.333 x 3
28 : 2.667 x 3
8 : 3.0 x 3
5 : 3.333 x 3
14 : 3.333 x 3
3 : 3.333 x 3
9 : 3.667 x 3
20 : 3.667 x 3
4 : 3.667 x 3
15 : 4.0 x 3
6 : 4.667 x 3
18 : 5.0 x 3
30 : 5.0 x 3
10 : 5.0 x 3
17 : 5.333 x 3
25 : 6.0 x 3
23 : 6.333 x 3
19 : 6.333 x 3
1 : 6.333 x 3
29 : 6.333 x 3
21 : 7.333 x 3
7 : 7.667 x 3
12 : 7.667 x 3
2 : 9.0 x 3
16 : 10.0 x 3


9 : 3.667 x 6
30 : 3.833 x 6
4 : 4.0 x 6
1 : 4.167 x 6
20 : 4.167 x 6
23 : 4.333 x 6
18 : 4.333 x 6
29 : 4.5 x 6
21 : 4.667 x 6
10 : 5.0 x 6
6 : 5.167 x 6
12 : 5.167 x 6
7 : 5.5 x 6
17 : 5.667 x 6
3 : 5.667 x 6
15 : 5.833 x 6
25 : 6.167 x 6
19 : 6.333 x 6
16 : 7.167 x 6
2 : 7.5 x 6


12 : 4.111 x 9
7 : 4.333 x 9
17 : 4.556 x 9
6 : 5.111 x 9
16 : 5.444 x 9
19 : 5.444 x 9
2 : 5.778 x 9
3 : 5.778 x 9
15 : 5.889 x 9
25 : 6.111 x 9


### 8) 
Play every rounds and display top 10 and podium

In [266]:
#generate players
n=10
tree=Classement()
root=manche1(n)
print("ROUND 1 :")
tree.inOrder(root)
#play round 2 to 10
for i in range(n-1):
    root=manche(root)
    print(f"\nROUND {i+2} : ")
    tree.inOrder(root)
#play final round
root=manche(root,nb_games=5,final=True)
#display top 10
print("\nTOP 10 after final : ")
tree.inOrder(root)
#display podium
print("\nPodium : ")
for _ in range(7):
    minPlayer=tree.getMinPlayer(root)
    root=tree.delete(root,minPlayer)
tree.inOrder(root)


ROUND 1 :
19 : 0.333 x 3
21 : 0.333 x 3
68 : 1.667 x 3
27 : 1.667 x 3
75 : 1.667 x 3
64 : 2.0 x 3
79 : 2.0 x 3
80 : 2.0 x 3
65 : 2.333 x 3
33 : 2.333 x 3
88 : 2.333 x 3
6 : 2.333 x 3
26 : 2.333 x 3
23 : 2.333 x 3
2 : 2.333 x 3
70 : 2.333 x 3
7 : 2.333 x 3
22 : 2.333 x 3
76 : 2.667 x 3
72 : 2.667 x 3
4 : 2.667 x 3
25 : 2.667 x 3
14 : 3.333 x 3
1 : 3.333 x 3
63 : 3.333 x 3
31 : 3.333 x 3
5 : 3.333 x 3
89 : 3.333 x 3
78 : 3.333 x 3
94 : 3.667 x 3
99 : 3.667 x 3
77 : 3.667 x 3
62 : 3.667 x 3
53 : 3.667 x 3
66 : 3.667 x 3
50 : 3.667 x 3
85 : 3.667 x 3
58 : 3.667 x 3
60 : 3.667 x 3
86 : 3.667 x 3
20 : 4.0 x 3
82 : 4.0 x 3
52 : 4.0 x 3
16 : 4.0 x 3
98 : 4.0 x 3
59 : 4.0 x 3
15 : 4.0 x 3
91 : 4.0 x 3
37 : 4.0 x 3
39 : 4.0 x 3
24 : 4.0 x 3
44 : 4.0 x 3
35 : 4.0 x 3
38 : 4.0 x 3
54 : 4.333 x 3
11 : 4.333 x 3
46 : 4.333 x 3
17 : 4.333 x 3
34 : 4.333 x 3
49 : 4.333 x 3
81 : 4.667 x 3
41 : 4.667 x 3
57 : 4.667 x 3
96 : 5.0 x 3
43 : 5.0 x 3
51 : 5.0 x 3
18 : 5.0 x 3
90 : 5.0 x 3
47 : 5.0 x 3
84 : 5.

## Part 2 :

In [267]:
%reset -f

In [268]:
import numpy as np
import time

In [269]:
## Construction of the adjacency matrix representing the graph
A = np.zeros((9,9));
A[0,1] = 1
A[0,5] = 1
A[1,2] = 1
A[1,6] = 1
A[2,3] = 1
A[2,7] = 1
A[3,8] = 1
A[4,6] = 1
A[4,7] = 1
A[5,7] = 1
A[5,8] = 1
A[6,8] = 1
A = A + np.transpose(A);

#List that represents the color of the edges in the coloration problem, we start by giving them all the "impostor" color.
joueursCol =[1,1,1,1,1,1,1,1,1]

#List of potential impostors (the alive players that saw player 0 before he got killed)
imp_poten=[1,4,5]

In [270]:
def Solution(A,joueursCol,imp_poten):
    solution = []
    for i in imp_poten:
        joueursCol1 = joueursCol.copy()   #Copying the list of colors so it resets every loop
        for j in range(len(joueursCol)):  #Here, we "colorize" the potential impostor - neighbours, as they can't be impostors 
            if A[i-1,j] == 1 :
                joueursCol1[j]=0
        for h in range(len(joueursCol)): #finally, we create a list of the possible impostors duo, for each "seed" potential impostor
            if joueursCol1[h]==1 and h != i-1:  #owe try to avoid getting tuples such as (1,1)
                solution+= [(i,h+1)]  
                
    #And finally, we print the three possible sets, depending on the seed impostor            
    print(u"The possible impostor set if player 1 killed player 0 is : \n S1 = " + " {" + str(solution[0:6]) + "}")
    print(u"The possible impostor set if player 4 killed player 0 is : \n S2 = " + " {" + str(solution[6:12]) + "}")
    print(u"The possible impostor set if player 5 killed player 0 is : \n S3 = " + " {" + str(solution[12:18]) + "}")
    return(solution[0:6],solution[6:12],solution[12:18])

Solution(A,joueursCol,imp_poten);

The possible impostor set if player 1 killed player 0 is : 
 S1 =  {[(1, 3), (1, 4), (1, 5), (1, 7), (1, 8), (1, 9)]}
The possible impostor set if player 4 killed player 0 is : 
 S2 =  {[(4, 1), (4, 2), (4, 5), (4, 6), (4, 7), (4, 8)]}
The possible impostor set if player 5 killed player 0 is : 
 S3 =  {[(5, 1), (5, 2), (5, 3), (5, 4), (5, 6), (5, 9)]}


## Part 3 :

In [271]:
%reset -f

In [272]:
import numpy as np
import time

In [273]:
# First of all, let's create a dict containing the names of the rooms,
# correspondings to their numbers 
dict_rooms = {1 : "Caféteria", 2 : "Weapons", 3 : "Navigations",
              4 : "O2", 5 : "Shield", 6 : "Communication",
              7 : "Storage", 8 : "Electrical",  9 : "Medbay",
              10 : "Upper E", 11 : "Lower E", 12 : "Reactor",
              13 : "Security", 14 : "Admin"}

# Then, let's create the adjacency matrix for the crewmates map's graph, using the mesure we took on the map
map_crewmates = np.zeros((14,14))    

map_crewmates[0,1] = 5.8
map_crewmates[0,6] = 8.3
map_crewmates[0,8] = 7.15
map_crewmates[0,9] = 9.8
map_crewmates[0,13] = 7.25
map_crewmates[1,2] = 7.15
map_crewmates[1,3] = 4.6
map_crewmates[1,4] = 8.8
map_crewmates[2,3] = 6.6
map_crewmates[2,4] = 7.8
map_crewmates[3,4] = 8.4
map_crewmates[4,5] = 4.1
map_crewmates[4,6] = 6.65
map_crewmates[5,6] = 5.5
map_crewmates[6,7] = 6.85
map_crewmates[6,10] = 9.85
map_crewmates[6,13] = 6.21
map_crewmates[7,10] = 7.7
map_crewmates[8,9] = 7.15
map_crewmates[9,10] = 7.3
map_crewmates[9,11] = 5.55
map_crewmates[9,12] = 5.1
map_crewmates[10,11] = 5.7
map_crewmates[10,12] = 5.2
map_crewmates[11,12] = 4.9

map_impostors = map_crewmates.copy()

map_crewmates += np.transpose(map_crewmates)


### replacing the values, adding the edges representing the new ways of moving using the vents

map_impostors[9,11]=2.8
map_impostors[10,11]=2.8
map_impostors[8,12]=2.6
map_impostors[7,8]=2.3
map_impostors[7,12]=2.5
map_impostors[0,13]=3.6
map_impostors[0,3]=7.1
map_impostors[0,4]=6
map_impostors[0,2]=6.8
map_impostors[2,13]=6
map_impostors[3,13]=6.3
map_impostors[4,13]=5.2
map_impostors[1,13]=3.6
map_impostors[2,4]=2.3
map_impostors[1,2]=2.6

map_impostors += np.transpose(map_impostors)

print("Map crewmates : \n",map_crewmates)
print("\nMap impostors : \n",map_impostors)

Map crewmates : 
 [[0.   5.8  0.   0.   0.   0.   8.3  0.   7.15 9.8  0.   0.   0.   7.25]
 [5.8  0.   7.15 4.6  8.8  0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   7.15 0.   6.6  7.8  0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   4.6  6.6  0.   8.4  0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   8.8  7.8  8.4  0.   4.1  6.65 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   4.1  0.   5.5  0.   0.   0.   0.   0.   0.   0.  ]
 [8.3  0.   0.   0.   6.65 5.5  0.   6.85 0.   0.   9.85 0.   0.   6.21]
 [0.   0.   0.   0.   0.   0.   6.85 0.   0.   0.   7.7  0.   0.   0.  ]
 [7.15 0.   0.   0.   0.   0.   0.   0.   0.   7.15 0.   0.   0.   0.  ]
 [9.8  0.   0.   0.   0.   0.   0.   0.   7.15 0.   7.3  5.55 5.1  0.  ]
 [0.   0.   0.   0.   0.   0.   9.85 7.7  0.   7.3  0.   5.7  5.2  0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   5.55 5.7  0.   4.9  0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   5.1  5.2  4.9  0.   0.  ]
 [7.25 0.   0.   0.   0.   0.   6

In [274]:
###In order to use Dijkstra and Floyd-Warshall, we first create two function, to convert an adjacency matrix in adjacency list, and vice-versa

#### Matrix to adjacency list 
def Mat_to_list(A):
    list_adj = {}     #creation d'un dictionnaire vide
    for i in range(len(A)):
        a=i+1                   # indice in the dict
        list_adj[a] = {}        # creation of one dict by room
        for j in range (len(A)):    # For every item in the matrix, we had the value to the corresponding dict
            if A[i,j] != 0:
                list_adj[a][j+1]=A[i,j] 
    return(list_adj)

B = Mat_to_list(map_crewmates)  # Convert crewmates and impostors graph's adjacency matrix in an adjacency list
C= Mat_to_list(map_impostors)
print("Map crewmates :")
[print(i) for i in B.items()]
print("\nMap impostors :")
[print(i) for i in C.items()];

Map crewmates :
(1, {2: 5.8, 7: 8.3, 9: 7.15, 10: 9.8, 14: 7.25})
(2, {1: 5.8, 3: 7.15, 4: 4.6, 5: 8.8})
(3, {2: 7.15, 4: 6.6, 5: 7.8})
(4, {2: 4.6, 3: 6.6, 5: 8.4})
(5, {2: 8.8, 3: 7.8, 4: 8.4, 6: 4.1, 7: 6.65})
(6, {5: 4.1, 7: 5.5})
(7, {1: 8.3, 5: 6.65, 6: 5.5, 8: 6.85, 11: 9.85, 14: 6.21})
(8, {7: 6.85, 11: 7.7})
(9, {1: 7.15, 10: 7.15})
(10, {1: 9.8, 9: 7.15, 11: 7.3, 12: 5.55, 13: 5.1})
(11, {7: 9.85, 8: 7.7, 10: 7.3, 12: 5.7, 13: 5.2})
(12, {10: 5.55, 11: 5.7, 13: 4.9})
(13, {10: 5.1, 11: 5.2, 12: 4.9})
(14, {1: 7.25, 7: 6.21})

Map impostors :
(1, {2: 5.8, 3: 6.8, 4: 7.1, 5: 6.0, 7: 8.3, 9: 7.15, 10: 9.8, 14: 3.6})
(2, {1: 5.8, 3: 2.6, 4: 4.6, 5: 8.8, 14: 3.6})
(3, {1: 6.8, 2: 2.6, 4: 6.6, 5: 2.3, 14: 6.0})
(4, {1: 7.1, 2: 4.6, 3: 6.6, 5: 8.4, 14: 6.3})
(5, {1: 6.0, 2: 8.8, 3: 2.3, 4: 8.4, 6: 4.1, 7: 6.65, 14: 5.2})
(6, {5: 4.1, 7: 5.5})
(7, {1: 8.3, 5: 6.65, 6: 5.5, 8: 6.85, 11: 9.85, 14: 6.21})
(8, {7: 6.85, 9: 2.3, 11: 7.7, 13: 2.5})
(9, {1: 7.15, 8: 2.3, 10: 7.15, 13: 2.6})

In [275]:
### List to adjacency Matrix
def List_to_Matrix(B):
    a = len(B) 
    mat_adj= np.zeros((a,a))
    for key,value in B.items() :    # For every item in the dict, we get the value and add it in the corresponding spot in the matrix
        for i in value :
            a,b = i
            mat_adj[key-1,a-1]=b
            
    return(mat_adj)

## Dijkstra
def Dijkstra(dg,s):
    infini = sum(sum(dg[sommet][s2]for s2 in dg[sommet]) for sommet in dg) +1  # Create an "infinite" value, that we are sure that no path can even get close to
    s_connu = {s : [0,[s]]}         ## distance from s to s is set to 0
    s_inconnu = {k : [infini, ''] for k in dg if k!=s}  ## distance from s to any other vertice is set to infinite
    for suivant in dg[s]:    #For every direct neighbour of s, we set the distance as being the weight of the path from s to said vertice
        s_inconnu[suivant] = [dg[s][suivant],s]  
    while s_inconnu and any(s_inconnu[k][0]<infini for k in s_inconnu):  #tant qu'il reste des elements dans s_inconnu de distance non infinie
        u = min(s_inconnu, key = s_inconnu.get)  #pick a vertice in set of unknow vertices and set the distance
        longueur_u, precedent_u = s_inconnu[u]
        for v in dg[u]:
            if v in s_inconnu:  #for every unknow vertice connected to u
                d = longueur_u + dg[u][v]  #Check for the path's weight
                if d<s_inconnu[v][0]:       # if it's smaller from the current path
                    s_inconnu[v] = [d,u]    # set it as the new distance/path
        s_connu[u] = [longueur_u,s_connu[precedent_u][1]+[u]]
        del s_inconnu[u]     #delete visited vertice
    return(s_connu)

## Dijkstra applied to every edge
def All_Shortest_Path_D(dg):
    start_time = time.time()   #timer in order to check which algorithm is the fastest
    a = {}
    for j in range (len(dg)):     #simply apply Dijkstra to every single vertice
        a[j+1] = Dijkstra(dg,j+1)
    print(time.time()-start_time)
    return(a)
           
INF = 9999


# Function in order to round up the value in the floyd-warshall distance matrix
def transformation(graph):
    l = len(graph)
    for i in range (l) :
        for j in range(l):
            if graph[i,j]==0:
                graph[i,j]=INF
    graph=np.around(graph,decimals=2)
    return(graph)
      
## Floyd-Warshall
def floyd_warshall(G):
    start_time = time.time()
    distance = list(map(lambda i: list(map(lambda j: j, i)), G))  #distance matrix
    nV=len(G)
    # Adding vertices individually
    for k in range(nV):         
        for i in range(nV):         
            for j in range(nV): 
                distance[i][j] = min(distance[i][j], distance[i][k] + distance[k][j])  #Comparing the distances and picking the smallest one
    print(time.time()-start_time)
    return(np.around(distance,decimals=1))

In [276]:
def Exercice3():
    mat = np.zeros((14,14))
    mat1 = np.zeros((14,14))
    mat2=np.zeros((14,14))
    a = All_Shortest_Path_D(B)
    b = All_Shortest_Path_D(C)
    print("Crewmates times :\n")
    for cle,valeurs in a.items():  #For every room
        for key,value in valeurs.items() :  #Print the time it takes to move to another room as a crewmate
             print (u"The shortest path from " +str(cle) + " to " + str(key) +" as a crewmate takes " + str(np.around(value[0],decimals=2)) + " sec")
             mat[cle-1,key-1]=np.around(value[0],decimals=2) #Fill a matrix of distances
    
    print("\n\nImpostors times :\n")
    for cle,valeurs in b.items():       #Same as before but as impostors
        for key,value in valeurs.items() :
             print (u"The shortest path from " +str(dict_rooms[cle]) + " to " + str(dict_rooms[key]) +" as an impostor takes " + str(np.around(value[0],decimals=2))+ " sec")
             mat1[key-1,cle-1]=np.around(value[0],decimals=2) 
    print(mat1)
    for i in range(14):         
        for j in range(i,14):       #Fill the upper side of the matrix with crewmates travelling times to move from a room to another
            mat2[i,j]=mat[i,j]  
        for k in range(0,i+1):      #Fill the lower side of the matrix with impostors travelling times to move from a room to another
            mat2[i,k] = mat1[i,k]
    return(mat2)
    
            #Finally, compare the timings and list the difference of travelling time between crewmates and impostors

E3=Exercice3();

0.0010008811950683594
0.0030100345611572266
Crewmates times :

The shortest path from 1 to 1 as a crewmate takes 0 sec
The shortest path from 1 to 2 as a crewmate takes 5.8 sec
The shortest path from 1 to 9 as a crewmate takes 7.15 sec
The shortest path from 1 to 14 as a crewmate takes 7.25 sec
The shortest path from 1 to 7 as a crewmate takes 8.3 sec
The shortest path from 1 to 10 as a crewmate takes 9.8 sec
The shortest path from 1 to 4 as a crewmate takes 10.4 sec
The shortest path from 1 to 3 as a crewmate takes 12.95 sec
The shortest path from 1 to 6 as a crewmate takes 13.8 sec
The shortest path from 1 to 5 as a crewmate takes 14.6 sec
The shortest path from 1 to 13 as a crewmate takes 14.9 sec
The shortest path from 1 to 8 as a crewmate takes 15.15 sec
The shortest path from 1 to 12 as a crewmate takes 15.35 sec
The shortest path from 1 to 11 as a crewmate takes 17.1 sec
The shortest path from 2 to 2 as a crewmate takes 0 sec
The shortest path from 2 to 4 as a crewmate takes 4.6

In [277]:
def Impostors_Shorter(mat):
    for i in range(len(mat)):
        for j in range(i,len(mat)):
            if mat[i,j]>mat[j,i]:
                print(u"Impostors take " + str(np.around((mat[i,j] - mat[j,i]),decimals =2))+ " secs less than crewmates to reach " + str(dict_rooms[j+1])+ " from " + str(dict_rooms[i+1]))
Impostors_Shorter(E3)

Impostors take 6.15 secs less than crewmates to reach Navigations from Caféteria
Impostors take 3.3 secs less than crewmates to reach O2 from Caféteria
Impostors take 8.6 secs less than crewmates to reach Shield from Caféteria
Impostors take 3.7 secs less than crewmates to reach Communication from Caféteria
Impostors take 5.7 secs less than crewmates to reach Electrical from Caféteria
Impostors take 2.15 secs less than crewmates to reach Lower E from Caféteria
Impostors take 2.75 secs less than crewmates to reach Reactor from Caféteria
Impostors take 5.15 secs less than crewmates to reach Security from Caféteria
Impostors take 3.65 secs less than crewmates to reach Admin from Caféteria
Impostors take 4.55 secs less than crewmates to reach Navigations from Weapons
Impostors take 3.9 secs less than crewmates to reach Shield from Weapons
Impostors take 3.9 secs less than crewmates to reach Communication from Weapons
Impostors take 4.29 secs less than crewmates to reach Storage from Weapon

## Part 4

In [278]:
%reset -f

<img src="map.png">

In [279]:
#aggency matrix :
graph=[
 [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0], #0
 [1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], #1
 [0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], #2
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #3
 [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], #4
 [0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0], #5
 [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0], #6
 [1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0], #7
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0], #8
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1], #9
 [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], #10
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1], #11
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1], #12
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0]] #13


#validation
for i in range(14):
    for j in range(14):
        if graph[i][j]!=graph[j][i]:
            print (i,j)

In [280]:
#return index of all vertices
def index(graph):
    return [sum(v) for v in graph]
index(graph)


[5, 5, 3, 2, 3, 5, 2, 6, 2, 5, 2, 3, 4, 3]

In [281]:
#graph : the aggency matrix of the graph
#start : the current vertice
#used : all the used vertices in order of use

def brutForce(graph,start,used):
    if len(used)==14 or sum(graph[start])-sum([graph[start][u] for u in used])==0: #if 
        if len(used)==14:
            print(used)
    else :
        for i in range(14) :
            if i not in used and graph[start][i]:
                u=[i for i in used]
                u.append(i)
                brutForce(graph,i,u)
for k in range(14):
    brutForce(graph,k,[k])

[3, 1, 2, 4, 5, 6, 7, 8, 9, 11, 13, 12, 0, 10]
[3, 1, 2, 4, 5, 6, 7, 8, 9, 13, 11, 12, 0, 10]
[3, 1, 2, 4, 5, 6, 7, 10, 0, 12, 11, 13, 9, 8]
[3, 1, 2, 4, 5, 6, 7, 10, 0, 12, 13, 11, 9, 8]
[3, 1, 4, 2, 5, 6, 7, 8, 9, 11, 13, 12, 0, 10]
[3, 1, 4, 2, 5, 6, 7, 8, 9, 13, 11, 12, 0, 10]
[3, 1, 4, 2, 5, 6, 7, 10, 0, 12, 11, 13, 9, 8]
[3, 1, 4, 2, 5, 6, 7, 10, 0, 12, 13, 11, 9, 8]
[6, 5, 2, 4, 1, 3, 0, 10, 7, 8, 9, 11, 12, 13]
[6, 5, 2, 4, 1, 3, 0, 10, 7, 8, 9, 11, 13, 12]
[6, 5, 2, 4, 1, 3, 0, 10, 7, 8, 9, 12, 11, 13]
[6, 5, 2, 4, 1, 3, 0, 10, 7, 8, 9, 12, 13, 11]
[6, 5, 2, 4, 1, 3, 0, 10, 7, 8, 9, 13, 11, 12]
[6, 5, 2, 4, 1, 3, 0, 10, 7, 8, 9, 13, 12, 11]
[6, 5, 2, 4, 1, 3, 0, 12, 11, 13, 9, 8, 7, 10]
[6, 5, 2, 4, 1, 3, 0, 12, 13, 11, 9, 8, 7, 10]
[6, 5, 4, 2, 1, 3, 0, 10, 7, 8, 9, 11, 12, 13]
[6, 5, 4, 2, 1, 3, 0, 10, 7, 8, 9, 11, 13, 12]
[6, 5, 4, 2, 1, 3, 0, 10, 7, 8, 9, 12, 11, 13]
[6, 5, 4, 2, 1, 3, 0, 10, 7, 8, 9, 12, 13, 11]
[6, 5, 4, 2, 1, 3, 0, 10, 7, 8, 9, 13, 11, 12]
[6, 5, 4, 2, 